In [1]:
# Q-Learning in FrozenLake

In [2]:
import numpy as np
import gymnasium as gym
import session_info

In [3]:
# Q-Learning parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.9  # Epsilon-greedy policy
episodes = 1000  # Number of episodes

In [4]:
# Initialize the environment
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False)

# Initialize Q-table
Q = np.zeros((env.observation_space.n, env.action_space.n))

def choose_action(state):
    if np.random.uniform(0, 1) < epsilon:
        return env.action_space.sample()  # Explore
    else:
        return np.argmax(Q[state, :])  # Exploit

In [5]:
# Q-Learning algorithm - Commens changes from SARSA example
for episode in range(episodes):
    state, _ = env.reset()
    done = False
    while not done:
        action = choose_action(state)  # Action selection moved here for Q-learning
        
        next_state, reward, done, _, _ = env.step(action)
        
        # Q-learning update
        # The key difference is using max Q-value of next state instead of Q-value of next action
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])
        
        state = next_state
        # Removed: action = next_action (Q-learning is off-policy, so we don't need to track the next action)

In [6]:
# Test the learned policy
def test_policy(n_episodes=100):
    successes = 0
    for _ in range(n_episodes):
        state, _ = env.reset()
        done = False
        while not done:
            action = np.argmax(Q[state, :])
            state, reward, done, _ , _ = env.step(action)
            if reward == 1:
                successes += 1
    return successes / n_episodes

success_rate = test_policy()
print(f"Success rate: {success_rate:.2%}")

# Display the learned Q-table
print("\nLearned Q-table:")
print()
print(Q)


Success rate: 100.00%

Learned Q-table:

[[0.86110453 0.88150032 0.83803281 0.86504857]
 [0.8532845  0.         0.78924798 0.82592581]
 [0.80517106 0.8214873  0.68447934 0.74719526]
 [0.74417055 0.         0.62724198 0.60213811]
 [0.86952671 0.89469847 0.         0.85568931]
 [0.         0.         0.         0.        ]
 [0.         0.87485126 0.         0.57207764]
 [0.         0.         0.         0.        ]
 [0.88138096 0.         0.91284037 0.85175735]
 [0.79181551 0.87003423 0.93695088 0.        ]
 [0.77411446 0.96276636 0.         0.66104642]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.77725363 0.92192218 0.76750054]
 [0.71202498 0.88829633 0.99030226 0.82088922]
 [0.         0.         0.         0.        ]]


In [7]:
# Testing the results

# Print learned policy
print("Learned Policy:")
print("===============")
print()
policy = np.argmax(Q, axis=1)
policy_symbols = ['←', '↓', '→', '↑', 'S', 'G', 'H']  # Up, Right, Down, Left

# Mark special positions
desc = env.unwrapped.desc
for i in range(4):
    for j in range(4):
        if desc[i][j] == b'H':
            policy[i*4+j%4] = 6
        elif desc[i][j] == b'G':
            policy[i*4+j%4] = 5
        elif desc[i][j] == b'S':
            policy[i*4+j%4] = 4

for i in range(4):
    for j in range(4):
        print(policy_symbols[policy[i*4 + j]], end=' ')
    print()
  
    

# Print value function
print("\nValue Function:")
print("===================")
V = np.max(Q, axis=1)
for i in range(4):
    for j in range(4):
        print(f"{V[i*4 + j]:.2f}", end=' ')
    print()


# Test the learned policy
env = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
state, _ = env.reset()
state = int(state)
done = False
total_reward = 0

while not done:
    action = np.argmax(Q[state, :])
    state, reward, terminated, truncated, _ = env.step(action)
    state = int(state)
    done = terminated or truncated
    total_reward += reward
    env.render()

print(f"\nTotal reward: {total_reward}")

Learned Policy:

S ← ↓ ← 
↓ H ↓ H 
→ → ↓ H 
H → → G 

Value Function:
0.88 0.85 0.82 0.74 
0.89 0.00 0.87 0.00 
0.91 0.94 0.96 0.00 
0.00 0.92 0.99 0.00 

Total reward: 1.0


In [8]:
session_info.show(html=False)

-----
gymnasium           0.29.1
numpy               1.26.4
session_info        1.0.0
-----
IPython             8.26.0
jupyter_client      8.6.2
jupyter_core        5.7.2
-----
Python 3.12.3 (main, Sep 11 2024, 14:17:37) [GCC 13.2.0]
Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.39
-----
Session information updated at 2024-09-19 16:50
